In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn
import numpy as np
import vaex
import os

In [3]:
path = os.path.join("..", "data", "processed", "test", "test_1.arrow")
df = vaex.open(path)
df

#,proteinA_seq,proteinB_seq,proteinA_x,proteinB_x,proteinA_y,proteinB_y,proteinA_z,proteinB_z,proteinA_hydropathy,proteinB_hydropathy,proteinA_hydropathy_index,proteinB_hydropathy_index,proteinA_acidity_basicity,proteinB_acidity_basicity,proteinA_mass,proteinB_mass,proteinA_isoelectric_point,proteinB_isoelectric_point,proteinA_charge,proteinB_charge,interaction
0,[ 6 10 3 ... 0 0 0],[ 6 15 15 ... 0 0 0],[-34. -31. -31. ... 0. 0. 0.],[13. 17. 14. ... 0. 0. 0.],[-21. -20. -18. ... 0. 0. 0.],[15. 14. 9. ... 0. 0. 0.],[23. 23. 18. ... 0. 0. 0.],[-65. -68. -69. ... 0. 0. 0.],[2 2 1 ... 0 0 0],[2 1 1 ... 0 0 0],[-0.4 3.8 -3.5 ... 0. 0. 0. ],[-0.4 -4.5 -4.5 ... 0. 0. 0. ],[0 0 1 ... 0 0 0],[0 2 2 ... 0 0 0],[ 75.07 131.18 133.1 ... 0. 0. 0. ],[ 75.07 174.2 174.2 ... 0. 0. 0. ],[5.97 5.98 2.77 ... 0. 0. 0. ],[ 5.97 11.15 11.15 ... 0. 0. 0. ],[0 0 2 ... 0 0 0],[0 1 1 ... 0 0 0],[1 0]
1,[ 6 15 15 ... 0 0 0],[ 6 10 3 ... 0 0 0],[13. 17. 14. ... 0. 0. 0.],[-34. -31. -31. ... 0. 0. 0.],[15. 14. 9. ... 0. 0. 0.],[-21. -20. -18. ... 0. 0. 0.],[-65. -68. -69. ... 0. 0. 0.],[23. 23. 18. ... 0. 0. 0.],[2 1 1 ... 0 0 0],[2 2 1 ... 0 0 0],[-0.4 -4.5 -4.5 ... 0. 0. 0. ],[-0.4 3.8 -3.5 ... 0. 0. 0. ],[0 2 2 ... 0 0 0],[0 0 1 ... 0 0 0],[ 75.07 174.2 174.2 ... 0. 0. 0. ],[ 75.07 131.18 133.1 ... 0. 0. 0. ],[ 5.97 11.15 11.15 ... 0. 0. 0. ],[5.97 5.98 2.77 ... 0. 0. 0. ],[0 1 1 ... 0 0 0],[0 0 2 ... 0 0 0],[0 1]
2,[ 6 10 3 ... 0 0 0],[ 6 15 15 ... 0 0 0],[-34. -31. -31. ... 0. 0. 0.],[13. 17. 14. ... 0. 0. 0.],[-21. -20. -18. ... 0. 0. 0.],[15. 14. 9. ... 0. 0. 0.],[23. 23. 18. ... 0. 0. 0.],[-65. -68. -69. ... 0. 0. 0.],[2 2 1 ... 0 0 0],[2 1 1 ... 0 0 0],[-0.4 3.8 -3.5 ... 0. 0. 0. ],[-0.4 -4.5 -4.5 ... 0. 0. 0. ],[0 0 1 ... 0 0 0],[0 2 2 ... 0 0 0],[ 75.07 131.18 133.1 ... 0. 0. 0. ],[ 75.07 174.2 174.2 ... 0. 0. 0. ],[5.97 5.98 2.77 ... 0. 0. 0. ],[ 5.97 11.15 11.15 ... 0. 0. 0. ],[0 0 2 ... 0 0 0],[0 1 1 ... 0 0 0],[1 0]


In [5]:
from anu.models.cnn.loader import InteractionClassificationDataset

In [6]:
from torch.utils.data import DataLoader

In [7]:
int_dataset = InteractionClassificationDataset(path)

In [8]:
int_dataloader = DataLoader(int_dataset, batch_size=2, shuffle=True, num_workers=0)

In [21]:
part2 = nn.Sequential(
            # 2D convolution layer
            nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 2D convolution layer
            nn.Conv2d(10, 30, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(30),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 2D convolution layer
            nn.Conv2d(30, 30, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(30),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
)

In [23]:
fcn = nn.Sequential(
            nn.Linear(30 * 1 * 500, 1500),
            nn.ReLU(inplace=True),
            nn.Linear(1500, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 12),
            nn.ReLU(inplace=True),
            nn.Linear(12, 2),
        )

[autoreload of anu.models.cnn.model failed: Traceback (most recent call last):
  File "/Volumes/HDD/packages/anaconda2/envs/anu/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Volumes/HDD/packages/anaconda2/envs/anu/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Volumes/HDD/packages/anaconda2/envs/anu/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Volumes/HDD/packages/anaconda2/envs/anu/lib/python3.8/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'anu.models.cnn' not in sys.modules
]


In [25]:
for labels, inputs in int_dataloader:
    output = part2(inputs.float())
    output2 = fcn(output.view(output.size(0), -1))
    print(output.shape)
    print(output)
    print(output2.shape)
    print(output2)
    break

torch.Size([2, 30, 1, 500])
tensor([[[[4.4932, 2.9787, 3.2913,  ..., 0.1757, 0.1757, 0.1836]],

         [[1.9308, 0.4355, 0.1256,  ..., 0.2395, 0.2395, 0.2346]],

         [[8.2406, 6.5865, 5.9405,  ..., 0.0429, 0.0429, 0.1640]],

         ...,

         [[6.5146, 5.6253, 6.3229,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.2617, 0.2617, 0.2891]],

         [[6.1534, 1.0316, 1.0018,  ..., 0.1710, 0.1710, 0.4959]]],


        [[[3.5668, 3.5741, 2.9892,  ..., 0.1757, 0.1757, 0.1836]],

         [[1.7045, 1.4150, 1.6701,  ..., 0.2395, 0.2395, 0.2346]],

         [[5.7255, 0.9436, 1.0516,  ..., 0.0429, 0.0429, 0.1640]],

         ...,

         [[4.6315, 5.1016, 4.8884,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.2617, 0.2617, 0.2891]],

         [[0.0000, 1.1325, 1.4612,  ..., 0.1710, 0.1710, 0.4959]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
torch.Size([2, 2])
tensor([[-0.0422,  0.3368],
        [-0.0043,  0.3218]], 

In [9]:
a, b = int_dataset[0]


In [31]:
loss = nn.CrossEntropyLoss()(output2, torch.autograd.Variable(targets)); loss

tensor(0.6954, grad_fn=<NllLossBackward>)

In [32]:
labels.max(dim=0)[1]

tensor([1, 0])

In [30]:
torch.autograd.Variable(targets)

tensor([1, 0])

In [34]:
from anu.models.cnn.trainer import CNNTrainer

In [35]:
int_dataloader

In [38]:
cnn_trainer = CNNTrainer(int_dataloader, int_dataloader)


In [ ]:
cnn_trainer.train()
